In [ ]:
from processingQst import *
from tqdm import tqdm
from operator import length_hint
import pandas as pd
import json

In [ ]:
df = pd.read_csv("../data/csv/train.csv",delimiter=";")
fragP1 = []
Qst = df["question"]
nbrRepAll = df["nbr_correct_answers"]
RepListIndxAll = df["correct_answers"]
# dfs = []
data_list = []

In [ ]:
for cpt in tqdm(range(len(Qst))):
# for cpt in range(len(Qst)):
    # if (cpt> 629):
    #     break
    if (cpt>=0):
        nbrRep = nbrRepAll[cpt]
        RepListIndx = RepListIndxAll[cpt]
        if (nbrRep!=1):
            RepListIndx= RepListIndx.replace('|', '') 
        qst = Qst[cpt]
        nwQst = ''
        listMed = []
        listMedAll = []
        ListMedAns = ''
        Spword = False

        # Détécter la négation 
        if(isRequestWrongAns(qst)):
            WrongQst = True 
            # print(WrongQst)
        else: 
            WrongQst = False 
            # print(WrongQst)
        
        # supprimer les parenthèses 
        qst = del_betParenthese(qst)
        # fragementation des questions selon les ","
        fragQst = splitQst(qst)
        for icpt in range(len(fragQst)):
            # Récupérer les fragements qui contiennent les infos médicales uniquement 
            if not (fragQst[icpt]==''):
                nwQst,Spword,listMed = recoverMedFrag(fragQst,icpt,nwQst,Spword)
                listMedAll.extend(listMed)
        nwQst = nwQst.lstrip()
        # print(listMedAll)
        # collect total pour impression dans un fichier 
        nwQstp = ''
        for j in range(nbrRep):
            Ans = df[f"answers.{RepListIndx[j]}"][cpt] 
            Ans = Ans.lstrip()
            listMed = RecoverListMed(Ans)
            # listMedAll.extend(listMed)
            listMedAll = list(set(listMedAll + listMed))
            # print(listMedAll)
            if Spword:
                nwQstp = replaceAtFront(Ans) + nwQst + '.'
            else:
                nwQstp = nwQst + ' '+ Ans.lower() + '.' 
            # Appliquer la négation 
            if WrongQst == True:
                nwQstp = appliquer_negation(nwQstp.lower())
            # créer un dataframe pour les données actuelles
            data_dict = {'keywords': listMedAll, 'sentence': nwQstp}
            data_list.append(data_dict)
            # data = pd.DataFrame({'ListMed': [','.join(listMedAll)], 'qst': [nwQstp]})
            # dfs.append(data)
            nwQstp = ''
            ListMedAll = ''

In [ ]:
# concaténer tous les dataframes dans un seul dataframe
# result = pd.concat(dfs, ignore_index=True)

# écrire les données dans un fichier CSV
# result.to_csv('./output/CorpusMS.csv', sep=';', index=False)
writeJson("output/corpusMS.json",data_list)